In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import os

path = "../temp/NetCDF_n/"
path_r = "../data/NSRDB_4km.nc"
files = os.listdir(path)
if ".DS_Store" in files: files.remove(".DS_Store")

In [5]:
xr.open_dataset(path + files[0])


<xarray.Dataset> Size: 6GB
Dimensions:                                      (time: 219144, lon: 81, lat: 3)
Coordinates:
  * time                                         (time) datetime64[ns] 2MB 19...
  * lon                                          (lon) float64 648B -100.5 .....
  * lat                                          (lat) float64 24B 19.41 ... ...
Data variables: (12/14)
    Temperature                                  (time, lat, lon) float64 426MB ...
    Dew Point                                    (time, lat, lon) float64 426MB ...
    DNI                                          (time, lat, lon) float64 426MB ...
    DHI                                          (time, lat, lon) float64 426MB ...
    GHI                                          (time, lat, lon) float64 426MB ...
    Relative Humidity                            (time, lat, lon) float64 426MB ...
    ...                                           ...
    Pressure                                     (time, lat, lon) float64 426MB ...
    Surface Albedo                               (time, lat, lon) float64 426MB ...
    Wind Direction                               (time, lat, lon) float64 426MB ...
    Wind Speed                                   (time, lat, lon) float64 426MB ...
    Global Horizontal UV Irradiance (280-400nm)  (time, lat, lon) float64 426MB ...
    Global Horizontal UV Irradiance (295-385nm)  (time, lat, lon) float64 426MB ...
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Tue May 28 05:13:35 2024: cdo -s -O collgrid /Volumes/DATA/...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...

In [2]:
ds_i = []
for f in files: ds_i.append( xr.open_dataset(path + f) )

vars = list(ds_i[0].keys())

ds = xr.Dataset( coords = { "time": ds_i[0]["time"], "lon": ds_i[0]["lon"],
    "lat": np.concatenate( [ x["lat"].values for x in ds_i ] ) } )

for v in vars:
    ds[v] = ( ("time", "lat", "lon"),
        np.concatenate( [ x[v].values for x in ds_i], axis = 1 ) )

ds.to_netcdf(path_r)

: 

In [ ]:
# Benchmarks
# 2
# concat: 00:09:17
# numpy: 00:09:10
# cdo: 00:02:03
# 3
# cdo: 00:03:04
# 2to4
# concat
# 3
# concat
# numpy

# Tamaño
# 1: 10.33 GB
# 2: 9.94 GB
# 4

In [3]:
ds = []
for f in files: ds.append( xr.open_dataset(path + f) )
xr.concat(ds, dim = "lat").to_netcdf(path_r)

: 

In [ ]:
ds = []
for f in files: ds.append( xr.open_dataset(path + f) )
xr.concat(ds, dim = "lat").to_netcdf(path_r)

In [1]:
# Convierte de CSV a NetCDF.

import os
import sys

import numpy as np
import pandas as pd

import xarray as xr

# Datos de entrada
path_d = "/Volumes/DATA/temp/temp/NSRDB_4km/CSV/"
path_r = "/Volumes/DATA/temp/temp/NSRDB_4km/NetCDF/"
files = os.listdir(path_d)
if ".DS_Store" in files: files.remove(".DS_Store")

In [2]:
f="20.33_-97.22.csv"
lat = f[:5]
lon = f[6:-4]
print(f" Procesando coordenadas {lat}°N {lon}°W...    ", end = "\r")

if not os.path.exists(f"{path_r}{lat}_{lon}.nc"):
    df = pd.read_csv( path_d + f, index_col = "time",
        parse_dates = True ).reset_index()

    # Convertimos a Dataset.
    df["lat" ] = float(lat)
    df["lon"] = float(lon)
    ds = df.set_index( ["time", "lat", "lon"] ).astype( float
        ).round( decimals = 1 ).to_xarray()
    ds["lat"] = ds["lat"].assign_attrs( standard_name = "latitude",
            long_name = "Latitude", units = "degrees" )
    ds["lon"] = ds["lon"].assign_attrs( standard_name = "longitude",
            long_name = "Longitude", units = "degrees" )
    ds.to_netcdf( f"{path_r}{lat}_{lon}.nc" )